# Simpleksna metoda

Sledeči program po korakih izvaja simpleksno metodo.

In [ ]:
def spremenljivke(n, zacetek=0, predpona="x", ciljna="z"):
    """
    Nastavi želeno število spremenljivk s podano predpono.
    """
    var(" ".join("%s%d" % (predpona, i) for i in range(zacetek, n+1)))
    var(ciljna)

def izpisi_enacbe(enacbe):
    """
    Uredi in ustrezno izpiše enačbe.
    """
    enacbe.sort(key=lambda e: str(e.lhs()))
    for e in enacbe:
        r = e.rhs()
        c = r.subs([v == 0 for v in r.variables()])
        rr = r - c
        s = str(rr)
        if c == 0:
            c = znak = ""
        elif s[0] == "-":
            s = s[1:]
            znak = " - "
        else:
            znak = " + "
        print("%s == %s%s%s" % (e.lhs(), c, znak, s))
    print("")
    return enacbe

def simpleksna_metoda(enacbe, *spremenljivke):
    """
    Za podani začetni slovar izvaja podane zamenjave baznih in nebaznih spremenljivk.
    """
    enacbe = izpisi_enacbe(enacbe)
    for vstopna, izstopna in spremenljivke:
        sol = next(e for e in enacbe if e.lhs() == izstopna).solve(vstopna)[0]
        enacbe = izpisi_enacbe([sol if e.lhs() == izstopna else e.subs(sol) for e in enacbe])

Funkcija `izpisi_enacbe` je pomožna. S funkcijo `spremenljivke` si pripravimo želeno število spremenljivk, funkcijo `simpleksna_metoda` pa uporabimo tako, da kot prvi argument podamo slovar enačb, za tem pa naštevamo pare vstopnih in izstopnih spremenljivk.

In [ ]:
spremenljivke(6)
simpleksna_metoda([
    x4 ==  5 - 2*x1 - 3*x2 -   x3,
    x5 == 11 - 4*x1 - 3*x2 -   x3,
    x6 ==  8 - 3*x1 - 4*x2 - 2*x3,
    z  ==      5*x1 + 4*x2 + 3*x3
], (x2, x4), (x3, x6))

Preverite, ali so vaše rešitve pravilne. Prav tako lahko poizkušate, kaj se zgodi, če izberete napačno vstopno ali izstopno spremenljivko.

# Dvofazna metoda

Pri prvi fazi dvofazne metode ugotavljamo dopustnost problema z n spremenljivkami tako, da rešujemo problem z $n+1$ spremenljivkami. Če je $n = 2$, si povezavo med problemoma lahko tudi predstavljamo.

In [ ]:
nastavitve = {'plot_points': 1000, 'incol': 'lightblue', 'bordercol': 'gray'}

In [ ]:
spremenljivke(5, 0, ciljna="w z")
omejitve = [
      x1 - x2 <= -1,
     -x1 - x2 <= -3,
    2*x1 + x2 <=  4
]
pozitivnost = [x1 >= 0, x2 >= 0, x0 >= 0]
meje = [(x1, -0.1, 3), (x2, -0.1, 7), (x0, 0, 3)]

In [ ]:
region_plot(omejitve + pozitivnost[:2], *meje[:2], **nastavitve)

In [ ]:
implicit_plot3d(max_symbolic(*([e.lhs() - e.rhs() - x0 for e in omejitve] +
                               [-e.lhs() for e in pozitivnost])), smooth=False, *meje)

In [ ]:
simpleksna_metoda([x == e.rhs() + x0 - e.lhs() for x, e in zip([x3, x4, x5], omejitve)] +
                  [w == -x0, z == x1 + x2], (x0, x4))

Za katere vrednosti parametra $a$ je sledeči problem dopusten? Pomagajte si z `@interact`.

\begin{align*}
\min &\ x + y \\
x + y &\ge a \\
2x + y &\ge 1 \\
x + 2y &\le 1 \\
x, y &\ge 0
\end{align*}

In [ ]:
x, y = var("x y")
meje = [(x, 0, 2), (y, 0, 2)]

@interact
def _(a=slider(0, 2, default=0, step_size=0.01, label='$a$'),
      k=slider(0, 2, default=1, step_size=0.01, label='$k$')):
    show(region_plot([x + y >= a, 2*x + y >= 1, x + 2*y <= 1], *meje, **nastavitve) +
         implicit_plot(x + y - k, *meje))

# Končnost simpleksne metode

Pri izbiri vstopnih in izstopnih spremenljivk upoštevaj naslednji pravili:

1. Vstopna spremenljivka naj bo tista, ki ima največji koeficient v vrstici, ki ustreza funkcionalu $z$.
2. Če imamo več kandidatov za izstopno spremenljivko, izberemo prvo po leksikografski ureditvi ($x1 < x2 < x3 < x4 < x5 < x6 < x7$).

Kaj opaziš?

In [ ]:
spremenljivke(7)
simpleksna_metoda([
    x5 == -1/2*x1 + 11/2*x2 + 5/2*x3 - 9*x4,
    x6 == -1/2*x1 + 3/2*x2 + 1/2*x3 - x4,
    x7 == 1 - x1,
    z == 10*x1 - 57*x2 - 9*x3 - 24*x4
], (x1, x5), (x2, x6))

Zgornji linearni program reši še s pomočjo Blandovega pravila (tako vstopne kot izstopne spremenljivke izbiraš glede na leksikografsko ureditev).

# Celoštevilsko linearno programiranje

* Poiščite optimalno rešitev sledečega problema:

\begin{align*}
\max \ 3x_1 + 4x_2 \\
2x_1 + x_2 &\le 6 \\
2x_1 + 3x_2 &\le 9 \\
x_1, x_2 &\ge 0
\end{align*}

* Nato poiščite še optimalno rešitev v primeru, da sta $x_1$ in $x_2$ celoštevilski spremenljivki.

* Nazadnje poiščite isto optimalno rešitev še na roke, torej tako, da ne izkoristite tega, da zna Sage reševati tudi celoštevilske probleme.

In [ ]:
def problem(integer=False):
    p = MixedIntegerLinearProgram(maximization=True)
    x = p.new_variable(integer=integer)
    p.set_objective(3*x[1] + 4*x[2])
    p.add_constraint(2*x[1] +   x[2] <= 6)
    p.add_constraint(2*x[1] + 3*x[2] <= 9)
    p.add_constraint(x[1] >= 0)
    p.add_constraint(x[2] >= 0)
    opt = p.solve()
    return (opt, p.get_values(x))

In [ ]:
# Linearni program
problem(integer=False)

In [ ]:
# Celoštevilski linearni program
problem(integer=True)

In [ ]:
spremenljivke(2)
meje = [(x1, 0, 3), (x2, 0, 3)]
tocke = sum(circle((i, j), 0.02, fill=True, rgbcolor='red') for i in range(meje[0][1], meje[0][2]+1)
                                                            for j in range(meje[1][1], meje[1][2]+1))

@interact
def _(k=slider(0, 15, default=0, step_size=0.01, label='$k$')):
    show(region_plot([2*x1 + x2 <= 6, 2*x1 + 3*x2 <=9], *meje, **nastavitve) + tocke +
         implicit_plot(3*x1 + 4*x2 - k, *meje))

# 0-1 nahrbtnik

Rešite problem 0-1 nahrbtnika z volumnom 10 in naslednjimi predmeti:

| volumen | cena |
| ------- | ---- |
| 2       | 30   |
| 4       | 60   |
| 6       | 50   |
| 3       | 40   |

In [ ]:
V = 10
v = [2, 4, 6, 3]
c = [30, 60, 50, 40]

resitev = [Set()] # seznam optimalnih rešitev pri dani omejitvi volumna
cena = [0] # seznam cen optimalnih rešitev pri dani omejitvi volumna
for i in range(1, V+1):
    r = resitev[-1]
    t = cena[-1]
    for j, (vj, cj) in enumerate(zip(v, c)):
        if vj > i or j in resitev[i - vj]:
            continue
        nova = cena[i - vj] + cj
        if nova > t:
            r = resitev[i - vj] | Set([j])
            t = nova
    resitev.append(r)
    cena.append(t)
    print("Volumen %d, rešitev %s, cena %d" % (i, r, t))